In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import sys,os,glob
from threading import Thread
from datetime import  datetime,timedelta
width = 13
base_link = "http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/09_2019/"

In [ ]:
class obrabotka_danniy(object):
#     def __init__(self,urls):
#         self.urls = urls
     
    @staticmethod
    def get_all_links_file(base_link=base_link):
        page = requests.get(base_link)
        soup = BeautifulSoup(page.content)
        result = []
        for link in soup.find_all('a', href=True):
            if (link['href'][-4]=='.' and link['href'][-1]=='B'):
                result.append(base_link + link['href'])
        return result

    @staticmethod
    def create_dir_for_downloading_files(self):
        path = os.path.abspath('') + '\\' + str(urls[0].split('/')[-2])
        try:
            os.path.isdir(path)
            os.mkdir(path)
        except FileExistsError as e:
            print(e)
            
    @staticmethod
    def download_files(urls=None):
        if urls==None:
            urls = ["http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/10_2019/10010008.19B",
                     "http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/10_2019/10010018.19B",
                       "http://amk030.imces.ru/meteodata/AMK_030_BIN/2019/06_2019/06010030.19R"]
        for i,j in enumerate(urls):
            handle = urllib.request.urlopen(urls[i])
            fname = os.path.basename(urls[i])
            with open(fname, "wb") as f_handler:
                while True:
                    chunk = handle.read()
                    if not chunk:
                        break
                    f_handler.write(chunk)
        
                    
    @staticmethod
    def get_file_names_in_dir(self):
        
        file_names = []
        file_name = os.path.abspath('')+'\\'
        for i in glob.glob("*.19B"):
            file_names.append(file_name + i)
        return file_names
    
    @staticmethod
    def function_obrabotka(default=0):
        data_frame_list = []
        file_names = os.path.abspath('')+'\\' + glob.glob("*.19B")[0]
        if default==1:
            file_names = get_file_names_in_dir()
        file_names = get_file_names_in_dir()
        for i,j in enumerate(file_names):
            with open(j, "rb") as file:
                content = file.read()
                data_dopolnitelniy = np.frombuffer(content[:17], dtype=np.dtype('i1'))
                data_dopolnitelniy = np.frombuffer(data_dopolnitelniy[:16], dtype=np.dtype('i2'))
                date_temp = data_dopolnitelniy[:-1]
                date_time = datetime(*date_temp).strftime("%d.%m.%Y %H:%M:%S.%f")
                numpy_data_polojitelnie = np.frombuffer(content, dtype=np.dtype('B'))[17:-14]
                numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie) / width), width)
                numpy_data_polojitelnie = np.delete(numpy_data_polojitelnie,12,1)
                numpy_data_polojitelnie = np.frombuffer(numpy_data_polojitelnie, dtype=np.dtype('i2'))
                numpy_data_polojitelnie = numpy_data_polojitelnie.reshape(int(len(numpy_data_polojitelnie)/6),6)
                row = numpy_data_polojitelnie.shape[0]
                column = numpy_data_polojitelnie.shape[1] + 1
                zero = np.zeros((row,1))
                numpy_data_polojitelnie = np.append(numpy_data_polojitelnie,zero,axis=1) 
                for i in range(len(numpy_data_polojitelnie)):
                    numpy_data_polojitelnie[i,0] = numpy_data_polojitelnie[i,0] / 100
                    numpy_data_polojitelnie[i,1] = numpy_data_polojitelnie[i,1] / 100
                    numpy_data_polojitelnie[i,2] = numpy_data_polojitelnie[i,2] / 100
                    numpy_data_polojitelnie[i,3] = numpy_data_polojitelnie[i,3] / 100
                    numpy_data_polojitelnie[i,4] = numpy_data_polojitelnie[i,4] / 10
                    numpy_data_polojitelnie[i,5] = numpy_data_polojitelnie[i,5] / 100
                datelist = pd.date_range(date_time, periods=len(numpy_data_polojitelnie), freq='12.5ms').to_pydatetime()
                data_frame = pd.DataFrame({'Time': datelist, 'Температура': numpy_data_polojitelnie[:,0], 'Южный компонент': numpy_data_polojitelnie[:,1],
                              'Восточный компонент': numpy_data_polojitelnie[:,2], 'Вертикальный компонент': numpy_data_polojitelnie[:,3],
                              'Атмосферное давление': numpy_data_polojitelnie[:,4], 'Влажность воздуха': numpy_data_polojitelnie[:,5],
                              'Признак ошибки': numpy_data_polojitelnie[:,6]})
                data_frame_list.append(data_frame)
        return data_frame_list


In [ ]:
func = obrabotka_danniy.download_files
links_files = obrabotka_danniy().get_all_links_file()
cut_links_files = [[links_files[:1000]],
                   [links_files[1000:2000]],
                   [links_files[2000:3000]],
                   [links_files[3000:]]]
cut_links_files = cut_links_files[0]

In [ ]:
for i,j in enumerate(cut_links_files):
    my_thread = Thread(target=func, name=[i], args=(j,))
    my_thread.start()

In [ ]:
def qwe():
    asd = obrabotka_danniy.function_obrabotka()
    asdf = pd.concat(asd)
    print(len(asdf))
    return asdf
asd = qwe()
